In [ ]:
%reset -f
import time
import pickle
import gym

# from stable_baselines3 import DDPG
# from stable_baselines3.common.env_checker import check_env
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import derivatives as d
import forward_helpers as fh
from chameleon import Chameleon

In [ ]:
n_elems = 100
steps = 1000
chameleon = Chameleon(E=100, n_elems=n_elems)

In [ ]:
# E = 1, 10 works E = 0.1, 100 breaks
# active_stress = np.array(
#     [
#         0.42846936,
#         0.47040876,
#         0.53477369,
#         0.62156414,
#         0.7307801,
#         0.86242159,
#         1.0164886,
#         1.19298113,
#         1.39189918,
#         1.61324275,
#     ]
# )

In [ ]:
fh.forward_simulate(chameleon, active_stress, sim_steps=steps)